<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/SVM_Whole_Brain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whole Brain Support Vector Machine Training
- Go to 'Runtime' in Colab browser bar, select 'Change Runtime Type', select 'High-RAM' from 'Runtime Shape'. 
- load local pickle file containing all masked, normalized Whole Brain subject data in numpy matrix format
- SVM training all subjects
- SVM training per subject

### Mount Google Drive and clone repository
- open to source directory

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount = True)

Mounted at /content/gdrive


In [3]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git
# Change directory into cloned repo
%cd teambrainiac/source
!ls


Cloning into 'teambrainiac'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (316/316), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 316 (delta 183), reused 240 (delta 117), pack-reused 0
Receiving objects: 100% (316/316), 57.14 MiB | 48.48 MiB/s, done.
Resolving deltas: 100% (183/183), done.
/content/teambrainiac/source
Access_Load_Data.ipynb		  percent_signal_change.ipynb
All_subject_masked_labeled.ipynb  SVM.ipynb
data				  SVM_Whole_Brain.ipynb
__init__.py			  utils.py
Masking.ipynb			  Visualize_Data.ipynb
Mat_to_Numpy.ipynb


### Load path_config.py 
- we are already in source so we can just load this file without chanding directory

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config.py
User uploaded file "path_config.py" with length 228 bytes


### Load the Whole brain normalized masked all subject 2d pickle file 
- go to Drive outside of this notebook
  - create a folder named data
  - upload 'whole_brain_all_norm_2d.pkl' - will take 5 hours but should remain on system without need to upload again
  - once uploaded, drag the file to the repo source/data directory

In [5]:
!ls

Access_Load_Data.ipynb		  path_config.py
All_subject_masked_labeled.ipynb  percent_signal_change.ipynb
data				  SVM.ipynb
__init__.py			  SVM_Whole_Brain.ipynb
Masking.ipynb			  utils.py
Mat_to_Numpy.ipynb		  Visualize_Data.ipynb


### Import libraries


In [6]:
# Import libraries
!pip install boto3 nilearn
import pickle
from utils import *
from sklearn.svm import SVC
import numpy as np
import random
import cv2 as cv

     |████████████████████████████████| 132 kB 27.9 MB/s 
     |████████████████████████████████| 10.1 MB 58.1 MB/s 
     |████████████████████████████████| 8.6 MB 61.5 MB/s 
     |████████████████████████████████| 79 kB 8.9 MB/s 
     |████████████████████████████████| 138 kB 78.7 MB/s 
     |████████████████████████████████| 127 kB 79.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


### Load Data from AWS and create Train/Val/Test splits

In [7]:
%%time
#pkl_file = "whole_brain_all_norm_2d.pkl" # normalized
pkl_file = 'all_data_dictionary.pkl' #Unnormalized
bool_mat = False
data = access_load_data(pkl_file, bool_mat)

In [9]:
#open path dictionary file to get subject ids
path = "data/data_path_dictionary.pkl"
data_path_dict = open_pickle(path)
subject_ids = data_path_dict['subject_ID']

# Randomly shuffle ids for train test val splits
random.seed(42)
random.shuffle(subject_ids)

In [10]:
# get train, test val data
train_ids = subject_ids[:36]
val_ids = subject_ids[36:44]
test_ids = subject_ids[44:53]

print(len(train_ids))
print(len(val_ids))
print(len(test_ids))
len(subject_ids)

36
8
8


52

In [11]:
# Get train test val data 
X_train = []
y_train = []

X_val = []
y_val = []

X_test = []
y_test = []

# Get X data from dictionary
for id_ in train_ids:
    for matrix in data[id_]:
        #print(matrix)
        X_train.append(matrix)
        
        
# Get y label from dictioanry
for id_ in train_ids:
    for label in data[f"{id_}_rt_labels"]:
        #print(matrix)
        y_train.append(label)
        


# Get X data from dictionary
for id_ in val_ids:
    for matrix in data[id_]:
        #print(matrix)
        X_val.append(matrix)
        
        
# Get y label from dictioanry
for id_ in val_ids:
    for label in data[f"{id_}_rt_labels"]:
        #print(matrix)
        y_val.append(label)
        
    

# Get X data from dictionary
for id_ in test_ids:
    for matrix in data[id_]:
        #print(matrix)
        X_test.append(matrix)
        
        
# Get y label from dictioanry
for id_ in test_ids:
    for label in data[f"{id_}_rt_labels"]:
        #print(matrix)
        y_test.append(label)
        
      

In [12]:
#4 runs * 52 subjects = 208
print(f"length of Xtrain data: {len(X_train)} and length of ytrain data: {len(y_train)}")
print(f"length of Xval data: {len(X_val)} and length of yval data: {len(y_val)}")
print(f"length of Xtest data: {len(X_test)} and length of ytest data: {len(y_test)}")
#print(f"length of X data: {len(X)} and length of y data: {len(y)}")

length of Xtrain data: 144 and length of ytrain data: 144
length of Xval data: 32 and length of yval data: 32
length of Xtest data: 32 and length of ytest data: 32


In [13]:
%%time
X_train = np.array(X_train)
y_train = np.array(y_train)
print( "Xtrain data shape ", X_train.shape)
print( "ytrain data shape ", y_train.shape)


Xtrain data shape  (144, 84, 237979)
ytrain data shape  (144, 84)
CPU times: user 1.47 s, sys: 2.37 s, total: 3.84 s
Wall time: 3.82 s


In [14]:
%%time
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)
print( "Xtrain data shape after concantenation ", X_train.shape)
print( "ytrain data shape after concantenation ", y_train.shape)

Xtrain data shape after concantenation  (12096, 237979)
ytrain data shape after concantenation  (12096,)
CPU times: user 753 ms, sys: 2.72 s, total: 3.47 s
Wall time: 3.45 s


## Set up SVM Model

#### SKlearn model

In [ ]:
%%time
clf = SVC()
clf.fit(X_train, y_train)

#### OpenCV library
- https://docs.opencv.org/3.4/d1/d73/tutorial_introduction_to_svm.html


In [ ]:
%%time

# Train the SVM using openCV
svm = cv.ml.SVM_create()
svm.setType(cv.ml.SVM_C_SVC)
svm.setKernel(cv.ml.SVM_LINEAR)
svm.setTermCriteria((cv.TERM_CRITERIA_MAX_ITER, 100, 1e-6))
svm.train(X_train, cv.ml.ROW_SAMPLE, y_train)
